In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/im-fault/2.0mm/28.0576.csv
/kaggle/input/im-fault/2.0mm/53.6576.csv
/kaggle/input/im-fault/2.0mm/37.6832.csv
/kaggle/input/im-fault/2.0mm/60.8256.csv
/kaggle/input/im-fault/2.0mm/47.5136.csv
/kaggle/input/im-fault/2.0mm/14.5408.csv
/kaggle/input/im-fault/2.0mm/57.344.csv
/kaggle/input/im-fault/2.0mm/56.32.csv
/kaggle/input/im-fault/2.0mm/49.5616.csv
/kaggle/input/im-fault/2.0mm/36.2496.csv
/kaggle/input/im-fault/2.0mm/34.2016.csv
/kaggle/input/im-fault/2.0mm/31.5392.csv
/kaggle/input/im-fault/2.0mm/51.8144.csv
/kaggle/input/im-fault/2.0mm/40.7552.csv
/kaggle/input/im-fault/2.0mm/24.1664.csv
/kaggle/input/im-fault/2.0mm/44.032.csv
/kaggle/input/im-fault/2.0mm/13.5168.csv
/kaggle/input/im-fault/2.0mm/54.4768.csv
/kaggle/input/im-fault/2.0mm/18.6368.csv
/kaggle/input/im-fault/2.0mm/55.9104.csv
/kaggle/input/im-fault/2.0mm/23.1424.csv
/kaggle/input/im-fault/2.0mm/42.5984.csv
/kaggle/input/im-fault/2.0mm/20.2752.csv
/kaggle/input/im-fault/2.0mm/50.3808.csv
/kaggle/input/im-fau

In [2]:
import pandas as pd
import glob
import os

# Paths to folders and their corresponding output values
folder_mappings = {
    '/kaggle/input/im-fault/0.5mm': 0,
    '/kaggle/input/im-fault/1.0mm': 0,
    '/kaggle/input/im-fault/1.5mm': 1,
    '/kaggle/input/im-fault/2.0mm': 1,
    '/kaggle/input/im-fault/normal': 0
}

# Parameters for downsampling
sample_fraction = 0.05  # Retain 10% of rows from each file

# Initialize an empty list to store DataFrames
dataframes = []

# Process each folder and its CSV files
for folder, output_value in folder_mappings.items():
    csv_files = glob.glob(os.path.join(folder, '*.csv'))
    
    for file in csv_files:
        # Read the file without assuming headers
        df = pd.read_csv(file, header=None)
        
        # Downsample the file
        downsampled_df = df.sample(frac=sample_fraction, random_state=42)
        
        # Add the output column
        downsampled_df['output'] = output_value
        
        # Append the modified DataFrame to the list
        dataframes.append(downsampled_df)

# Concatenate all the downsampled DataFrames
merged_df = pd.concat(dataframes, ignore_index=True)

# Add custom header columns
merged_df.columns = [
    'rotation freq', 'uh_ax_vib', 'uh_rd_vib', 'uh_tg_vib',
    'oh_ax_vib', 'oh_rd_vib', 'oh_tg_vib', 'microphone', 'output'
]

In [3]:
merged_df

,rotation freq,uh_ax_vib,uh_rd_vib,uh_tg_vib,oh_ax_vib,oh_rd_vib,oh_tg_vib,microphone,output
0,-0.69772,0.52789,-0.523310,-0.019556,-0.351490,-0.018541,-0.641970,0.212060,0
1,-0.65837,-0.23587,-0.053497,0.261080,0.772020,-0.037184,0.072600,-0.237030,0
2,-0.66380,-0.99015,-0.555890,-0.146720,0.155470,0.035417,0.297670,0.195460,0
3,4.39980,-0.42660,0.028898,0.076858,-0.361120,-0.039103,0.186340,0.217360,0
4,4.38420,-0.13118,0.038688,-0.388290,1.017300,0.020407,0.580360,0.122130,0
...,...,...,...,...,...,...,...,...,...
3074995,-0.64080,-0.16074,-0.173830,0.271070,0.059328,0.010774,0.446760,-0.210330,0
3074996,-0.55248,0.43913,0.363650,0.060409,-0.177610,-0.041612,-0.082842,-0.294850,0
3074997,-0.62519,-0.82865,0.282130,-0.127460,0.219360,0.047431,0.739000,0.329270,0
3074998,-0.72528,-0.65014,0.008187,-0.461870,0.141210,0.050601,0.082316,0.363710,0


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming 'merged_df' is already loaded and has the specified columns
# Features (input variables)
X = merged_df.drop(columns=['output'])

# Labels (target variable)
y = merged_df['output']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7230162601626017

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.87      0.79    370248
           1       0.72      0.50      0.59    244752

    accuracy                           0.72    615000
   macro avg       0.72      0.69      0.69    615000
weighted avg       0.72      0.72      0.71    615000

